In [ ]:
import os
import glob
import numpy as np # type: ignore
import pandas as pd # type: ignore
from boolean_functions import *
from xcslib import generate_all_bitstrings, match

In [2]:
def load_espresso_population(population_filepath):
    """Load the classifier population generated from an ESPRESSO solution
    which consists only of condition, action, and payoff."""
    with open(population_filepath,'r') as POPULATION:
        str_population = POPULATION.read()
    str_classifiers = [str_classifier.strip() for str_classifier in str_population.split('\n') if str_classifier.strip()!=""]    
    
    classifier_conditions = [str_classifier.split('\t')[0] for str_classifier in str_classifiers]
    classifier_actions = [str_classifier.split('\t')[1] for str_classifier in str_classifiers]
    classifier_payoffs = [int(str_classifier.split('\t')[2]) for str_classifier in str_classifiers]

    return pd.DataFrame(
    {"condition":classifier_conditions,
     "action":classifier_actions,
     "prediction":classifier_payoffs,
    })

In [ ]:
def convert_espresso_population_to_xcs(population_filepath, xcs_population_filepath, population_size):
    """convert a population in ESPRESSO format and convert it to the XCSLIB format of a given population size."""
    df_population = load_espresso_population(population_filepath)
    
    no_classifiers = len(df_population)
    
    print(no_classifiers)
    
    df_population['error'] = 0.0
    df_population['fitness'] = 1.0
    df_population['action_set_size'] = 100
    df_population['experience'] = 100

    num = population_size//no_classifiers
    numerosity = [num]*(no_classifiers-1)
    numerosity.append(population_size-sum(numerosity))

    df_population['numerosity'] = numerosity
    df_population['id'] = range(1,len(df_population)+1)

    df = df_population[['id','condition','action','prediction','error','fitness','action_set_size','experience','numerosity']].copy()
    
    df.to_csv(xcs_population_filepath, sep="\t", compression="gzip",index=False,header=False)    

In [ ]:
def evaluate_performance(df_populations, binary_function, verbose = False):
    """Evaluates a classifier population against a binary function by applying the initial XCS steps (matching and creation of the prediction array)"""

    rewards = []
    overlapping_niches = []
    
    n = len(df_population['condition'].values[0])
    
    if (verbose):
        print("# bits ",n)
    
    actions = np.unique(df_population["action"].values)
    
    if (verbose):
        print("actions ",str(actions))
        
    for binary_input in generate_all_bitstrings(n):

        correct_action = binary_function(binary_input)
        
        df_matching = df_populations[df_populations["condition"].apply(lambda condition: match(condition,binary_input))]

        prediction_array = np.zeros(len(actions))
        
        for action in actions:
            df_action_set = df_matching[df_matching["action"]==action]

            prediction_array[int(action)] = sum(df_action_set['prediction'].values)

        if (correct_action == np.argmax(prediction_array)):
            rewards.append(1000)
        else:
            rewards.append(0)

    return np.array(rewards).mean(),np.array(rewards).std()

## Evaluate Performance
Evaluate the performance of the generated populations on every possible combination.

In [ ]:
source_directory = "./populations/"
verbose = False

population_filepaths = glob.glob(source_directory + os.sep  + "*_classifiers.txt")
population_filepaths.sort()

for population_filepath in population_filepaths:

    filename = population_filepath.split(os.sep)[-1]
    print("Population: ",filename)

    if filename[:5]=="carry":
        binary_function = carry
    elif filename[:2]=="mp":
        binary_function = mp
    elif filename[:11]=="majority_on":
        binary_function = majority_on_action
    else:
        print(filename+"\n"+"Function not supported\n")
        continue

    print("loading ",population_filepath.split(os.sep)[-1])
    print("="*(1+len("loading "+population_filepath.split(os.sep)[-1])))
    
    df_population = load_espresso_population(population_filepath)

    mean_performance,std_performance = evaluate_performance(df_population,binary_function,True)

    prefix = population_filepath.split(os.sep)[-1].split("_")[0]

    print("%s - %5.3f %5.3f "%(prefix,mean_performance,std_performance))

Population:  carry10_classifiers.txt
loading  carry10_classifiers.txt
# bits  10
actions  ['0' '1']
carry10 - 1000.000 0.000 
Population:  carry12_classifiers.txt
loading  carry12_classifiers.txt
# bits  12
actions  ['0' '1']
carry12 - 1000.000 0.000 
Population:  carry6_classifiers.txt
loading  carry6_classifiers.txt
# bits  6
actions  ['0' '1']
carry6 - 1000.000 0.000 
Population:  carry8_classifiers.txt
loading  carry8_classifiers.txt
# bits  8
actions  ['0' '1']
carry8 - 1000.000 0.000 
Population:  majority_on10_classifiers.txt
loading  majority_on10_classifiers.txt
# bits  10
actions  ['0' '1']
majority - 1000.000 0.000 
Population:  majority_on11_classifiers.txt
loading  majority_on11_classifiers.txt
# bits  11
actions  ['0' '1']
majority - 1000.000 0.000 
Population:  majority_on12_classifiers.txt
loading  majority_on12_classifiers.txt
# bits  12
actions  ['0' '1']
majority - 1000.000 0.000 
Population:  majority_on6_classifiers.txt
loading  majority_on6_classifiers.txt
# bits 